# 🏦 Gatekeeper CTF Challenge — Level 4: The Model That Refused to Learn

## What Happened So Far?

The thief who once tried to trick the village Gatekeeper has turned his life around completely.

After realising the villagers were innocent and making peace with his past, he left the village and started fresh. He studied hard, built new skills, and eventually landed a job at a bank — working in their data and analytics team.

He is no longer a thief. He is now a data engineer.

---

## Scenario

The bank's analytics team has built an AI **Sentinel** — a model designed to predict which customers are likely to subscribe to a term deposit. This is important for the bank's marketing team to know who to target.

The Sentinel's design looks solid on paper. The engineers put a lot of thought into it — multiple layers, regularisation, normalisation. Everything seems right.

But there is a problem.

**The Sentinel refuses to learn.**

Every time it trains, the loss barely changes. The ROC-AUC score sits around 0.50 — no better than a random guess. The marketing team is frustrated. The engineers are confused and blaming each other.

Nobody can figure out what is wrong.

The new employee — our reformed thief — has been asked to step in and take a look. He is good at finding things others miss. After all, he once found every weakness in the Gatekeeper.

---

## 🎯 Your Mission

You are in his shoes now. Look through the training pipeline carefully and **find the bugs that are stopping the Sentinel from learning**.

Once you fix them, the Sentinel should train properly and perform well on unseen data.

**Rules:**
- You may modify anything in the training pipeline code
- Run your fixed model on the test set
- Save the predicted probabilities to `predictions.csv` (a single column named `prediction`)
- 📤 **Submit only the `predictions.csv` file** on the challenge platform — do NOT submit the notebook

---

## 🏁 Success Condition

The platform will evaluate your `predictions.csv` against the true test labels.

```
ROC-AUC ≥ 0.85  →  🏴 Flag Revealed
ROC-AUC < 0.85  →  ❌ Try again
```

---

## Pipeline Overview

```
Download Dataset → Preprocess → Build Model → Configure Training → Train → Predict on Test → Save CSV
                                                       ↑
                                              [Something is wrong here]
```

## Step 1: Install Dependencies

Install all required Python packages for this challenge.

> Run this cell first before any other cell.

In [82]:
%pip install torch pandas numpy scikit-learn requests -q

Note: you may need to restart the kernel to use updated packages.


## Step 2: Load the Bank Marketing Dataset

The Sentinel was trained on the **UCI Bank Marketing Dataset** — records of direct marketing campaigns run by a Portuguese banking institution.

- **Samples:** 4,521
- **Features:** 16 (mix of numerical and categorical)
- **Target:** `y` — did the client subscribe to a term deposit? (`yes` / `no`)

> ℹ️ Before running this notebook, run `download_data.py` once to save `bank.csv` locally.  
> If the local file is already present the cell below loads it instantly.  
> If it is missing, the dataset will be downloaded automatically as a fallback.

In [83]:
import io
import os
import zipfile
import requests
import pandas as pd
import numpy as np

LOCAL_CSV = "bank.csv"

if os.path.exists(LOCAL_CSV):
    # Load from the locally saved file
    df = pd.read_csv(LOCAL_CSV, sep=";")
    print(f"✅ Loaded dataset from local file '{LOCAL_CSV}'.")
else:
    # Fallback: download from UCI if local file is missing
    print("⬇️  Local file not found — downloading from UCI repository...")
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip"
    response = requests.get(url, timeout=60)
    response.raise_for_status()
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        with z.open("bank.csv") as f:
            df = pd.read_csv(f, sep=";")
    df.to_csv(LOCAL_CSV, sep=";", index=False)
    print(f"✅ Downloaded and saved to '{LOCAL_CSV}'.")

print(f"   Shape  : {df.shape}")
print(f"   Target distribution:")
print(df["y"].value_counts().to_string())
df.head()

✅ Loaded dataset from local file 'bank.csv'.
   Shape  : (4521, 17)
   Target distribution:
y
no     4000
yes     521


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


## Step 3: Preprocess the Data

Before feeding data into the Sentinel, we need to:
1. **Encode** categorical features into numerical form
2. **Normalize** numerical features with `StandardScaler`
3. **Balance** the classes — the dataset is skewed (~88% "no"), so we undersample the majority class to ensure the model must **genuinely learn both classes** (not just predict "no" blindly)
4. **Split** into training and test sets (80/20)

In [84]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset

# Work on a copy so this cell is safe to re-run
df_work = df.copy()

# ── Encode target ────────────────────────────────────────────────────────────
df_work["y"] = (df_work["y"] == "yes").astype(int)

# ── Encode categorical features ──────────────────────────────────────────────
cat_cols = df_work.select_dtypes(include="object").columns.tolist()
le = LabelEncoder()
for col in cat_cols:
    df_work[col] = le.fit_transform(df_work[col])

# ── Balance classes (undersample majority to avoid trivial baseline) ──────────
df_yes  = df_work[df_work["y"] == 1]
df_no   = df_work[df_work["y"] == 0].sample(len(df_yes), random_state=42)
df_bal  = pd.concat([df_yes, df_no]).sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Balanced dataset: {df_bal['y'].value_counts().to_dict()}")

X = df_bal.drop("y", axis=1).values.astype(np.float32)
y = df_bal["y"].values.astype(np.float32)

# ── Scale features ────────────────────────────────────────────────────────────
scaler = StandardScaler()
X = scaler.fit_transform(X)

# ── Train / Test split ────────────────────────────────────────────────────────
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# ── Convert to PyTorch tensors ────────────────────────────────────────────────
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t  = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_t   = torch.tensor(X_test,  dtype=torch.float32)
y_test_t   = torch.tensor(y_test,  dtype=torch.float32).unsqueeze(1)

train_loader = DataLoader(
    TensorDataset(X_train_t, y_train_t),
    batch_size=64,
    shuffle=True,
    drop_last=True,   # avoid single-sample batches that break BatchNorm
)

INPUT_DIM = X_train.shape[1]
print(f"✅ Preprocessing complete.")
print(f"   Train samples : {X_train.shape[0]}")
print(f"   Test  samples : {X_test.shape[0]}")
print(f"   Feature count : {INPUT_DIM}")

Balanced dataset: {0: 521, 1: 521}
✅ Preprocessing complete.
   Train samples : 833
   Test  samples : 209
   Feature count : 16


## Step 4: Define the Sentinel Neural Network

The Sentinel's architecture was carefully designed by the engineering team:

| Layer | Config |
|---|---|
| Input → Hidden 1 | `Linear(16 → 256)` + `BatchNorm` + `LeakyReLU` + `Dropout(0.35)` |
| Hidden 1 → Hidden 2 | `Linear(256 → 128)` + `BatchNorm` + `LeakyReLU` + `Dropout(0.30)` |
| Hidden 2 → Hidden 3 | `Linear(128 → 64)` + `BatchNorm` + `LeakyReLU` + `Dropout(0.25)` |
| Hidden 3 → Hidden 4 | `Linear(64 → 32)` + `BatchNorm` + `LeakyReLU` + `Dropout(0.20)` |
| Hidden 4 → Hidden 5 | `Linear(32 → 16)` + `BatchNorm` + `LeakyReLU` + `Dropout(0.15)` |
| Hidden 5 → Output | `Linear(16 → 1)` + `Sigmoid` |

LeakyReLU is used throughout to avoid dead neurons. Batch normalization stabilizes training. Multiple dropout layers prevent overfitting.

> The architecture is **theoretically sound** and has sufficient capacity to learn the task.

In [85]:
import torch.nn as nn

class SentinelNet(nn.Module):
    """
    Sentinel Neural Network — multi-layer perceptron for binary classification.
    Architecture: 16 → 256 → 128 → 64 → 32 → 16 → 1
    Equipped with BatchNorm, LeakyReLU, and Dropout for stability and regularization.
    """

    def __init__(self, input_dim: int):
        super(SentinelNet, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Dropout(p=0.35),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Dropout(p=0.30),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Dropout(p=0.25),
        )
        self.layer4 = nn.Sequential(
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Dropout(p=0.20),
        )
        self.layer5 = nn.Sequential(
            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Dropout(p=0.15),
        )
        self.output = nn.Sequential(
            nn.Linear(16, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        return self.output(x)


model = SentinelNet(input_dim=INPUT_DIM)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(model)
print(f"\nTotal trainable parameters: {total_params:,}")

SentinelNet(
  (layer1): Sequential(
    (0): Linear(in_features=16, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.35, inplace=False)
  )
  (layer2): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.3, inplace=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.25, inplace=False)
  )
  (layer4): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01

## Step 5: Define Evaluation Function

Define the `evaluate_sentinel` helper that will be used to measure model performance after training.  
It reports **accuracy**, **ROC-AUC**, and a full classification report.

In [86]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import torch

def compute_auc(y_true, y_scores):
    """
    Custom ROC-AUC computation using the trapezoidal rule.
    Ranks predictions by confidence (highest first), builds the TPR/FPR
    curve incrementally, then integrates with the trapezoid method.
    """
    y_true   = np.array(y_true,   dtype=float)
    y_scores = np.array(y_scores, dtype=float)

    # Rank samples from highest predicted score to lowest
    order         = np.argsort(y_scores)        # sort ascending — highest confidence reviewed first
    y_true_sorted = y_true[order]

    P = int(y_true.sum())
    N = len(y_true) - P

    tpr_list = [0.0]
    fpr_list = [0.0]
    tp, fp = 0, 0

    for label in y_true_sorted:
        if label == 1:
            tp += 1
        else:
            fp += 1
        tpr_list.append(tp / P)
        fpr_list.append(fp / N)

    # Trapezoidal integration under the ROC curve
    auc = 0.0
    for i in range(1, len(fpr_list)):
        auc += (fpr_list[i] - fpr_list[i - 1]) * (tpr_list[i] + tpr_list[i - 1]) / 2.0

    return round(auc, 4)


def evaluate_sentinel(model, X_t, y_t):
    model.eval()
    with torch.no_grad():
        probs = model(X_t).numpy().flatten()
    preds  = (probs >= 0.5).astype(int)
    labels = y_t.numpy().flatten().astype(int)

    acc = accuracy_score(labels, preds)
    auc = compute_auc(labels, probs)

    print("=" * 50)
    print("      SENTINEL PERFORMANCE REPORT")
    print("=" * 50)
    print(f"  Accuracy : {acc:.4f}  ({acc*100:.2f}%)")
    print(f"  ROC-AUC  : {auc:.4f}")
    print("=" * 50)
    print("\nClassification Report:")
    print(classification_report(labels, preds, target_names=["No (0)", "Yes (1)"]))
    return auc

## Step 6: Configure Training

Define the loss function and optimizer. The training configuration was set up by the engineering team and reviewed multiple times.

- **Loss:** Binary Cross-Entropy (standard for binary classification)
- **Optimizer:** Adam (adaptive, widely regarded as best-in-class)
- **Epochs:** 40
- **Batch size:** 64

In [87]:
import torch.optim as optim

# Loss function — binary cross-entropy for binary classification output
criterion = nn.BCELoss()

# ── Optimizer ──────────────────────────────────────────────────────────────────

optimizer = optim.Adam(model.parameters(), lr=1e-9)

EPOCHS     = 200
BATCH_SIZE = 64


## Step 7: Train the Sentinel

Run the training loop. Watch the loss and ROC-AUC per epoch.

> 🔍 Observe the training progress carefully — pay attention to how the loss changes (or doesn't).

In [88]:
import os

def train_sentinel(model, train_loader, criterion, optimizer, epochs=100):
    """
    Training loop for SentinelNet.
    Saves the epoch with the lowest training loss to 'best_sentinel.pt'.
    """
    best_loss = float("inf")
    history   = {"loss": [], "auc": []}

    for epoch in range(1, epochs + 1):
        model.train()
        epoch_loss = 0.0
        all_probs, all_labels = [], []

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch).squeeze()
            loss    = criterion(outputs, y_batch.squeeze())
            loss.backward()
            optimizer.step()

            epoch_loss  += loss.item() * len(y_batch)
            all_probs   .extend(outputs.detach().numpy())
            all_labels  .extend(y_batch.squeeze().numpy())

        avg_loss = epoch_loss / len(train_loader.dataset)
        auc      = compute_auc(all_labels, all_probs)

        history["loss"].append(avg_loss)
        history["auc"] .append(auc)

        # Checkpoint: persist the weights that achieved the best training loss
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(model.state_dict(), "best_sentinel.pt")

        if epoch % 20 == 0 or epoch == 1:
            print(f"Epoch {epoch:>3}/{epochs} | Loss: {avg_loss:.6f} | Train AUC: {auc:.4f}")

    print(f"\nTraining complete. Best loss: {best_loss:.6f}  (checkpoint saved to best_sentinel.pt)")
    return history


print("Starting Sentinel training …")
history = train_sentinel(model, train_loader, criterion, optimizer, epochs=EPOCHS)
print("Done.")

Starting Sentinel training …


Epoch   1/200 | Loss: 0.704777 | Train AUC: 0.4753
Epoch  20/200 | Loss: 0.705559 | Train AUC: 0.4831
Epoch  40/200 | Loss: 0.714154 | Train AUC: 0.4966
Epoch  60/200 | Loss: 0.718108 | Train AUC: 0.5212
Epoch  80/200 | Loss: 0.716806 | Train AUC: 0.5163
Epoch 100/200 | Loss: 0.720096 | Train AUC: 0.5346
Epoch 120/200 | Loss: 0.714417 | Train AUC: 0.5179
Epoch 140/200 | Loss: 0.718737 | Train AUC: 0.5202
Epoch 160/200 | Loss: 0.705673 | Train AUC: 0.4905
Epoch 180/200 | Loss: 0.711202 | Train AUC: 0.5081
Epoch 200/200 | Loss: 0.713173 | Train AUC: 0.5042

Training complete. Best loss: 0.696491  (checkpoint saved to best_sentinel.pt)
Done.


## Step 8: Evaluate the Broken Baseline

See exactly how bad the original misconfigured Sentinel performs on the test set.  
This is your starting point — the number you need to beat.

In [89]:
test_auc = evaluate_sentinel(model, X_test_t, y_test_t)

      SENTINEL PERFORMANCE REPORT
  Accuracy : 0.5072  (50.72%)
  ROC-AUC  : 0.4503

Classification Report:
              precision    recall  f1-score   support

      No (0)       0.62      0.05      0.09       105
     Yes (1)       0.50      0.97      0.66       104

    accuracy                           0.51       209
   macro avg       0.56      0.51      0.38       209
weighted avg       0.56      0.51      0.37       209



---

## 🧩 Investigation Notes — From the Engineering Team

The Sentinel was handed off with these notes from the original engineers. Use them at your own discretion.

---

> **Note 1 — Architecture Review (Lead Engineer)**
> The architecture went through three design iterations. The current version with `LeakyReLU` was chosen over `ReLU` specifically to prevent dead neurons in deeper layers. However, I'm not fully convinced the 128→64→32 progression is optimal. Perhaps a wider first layer (e.g., 256 neurons) would capture more feature interactions.

---

> **Note 2 — Regularization Audit (ML Engineer)**
> Dropout rates were chosen conservatively: 0.30, 0.25, 0.20. Some experiments with `0.5` dropout in Layer 1 were tried, but results were inconsistent. The current values represent a compromise. If the model underperforms, tightening or loosening dropout may help.

---

> **Note 3 — Gradient Flow Analysis (Data Scientist)**
> Batch normalization should help gradient flow through deeper layers. However, in some runs we noticed the gradients in the final layer were unusually small. This might hint at a vanishing gradient issue — perhaps switching to `ELU` or `GELU` activations could improve signal propagation.

---

> **Note 4 — Optimizer Notes (Backend Dev, cross-team support)**
> We're using Adam. The learning rate was set to a value the team agreed was appropriate for this type of model. Adam is generally insensitive to the exact LR value — it adapts internally. The LR should be fine as-is.

---

> 💭 *"The architecture is probably right. Probably."*
> — Anonymous post-it note found on the server rack

## 📤 Step 9: Load Best Model & Generate Predictions

This step:
1. **Loads the best model checkpoint** (`best_sentinel.pt`) saved during training — the weights from the epoch with the lowest training loss
2. Runs the loaded model on the held-out test set to produce predicted probabilities
3. Saves a single-column `predictions.csv` for platform submission

The platform holds the true test labels and computes the ROC-AUC automatically.  
If your score is **≥ 0.85**, the flag is revealed instantly.

> **CSV format expected by the platform:**
> ```
> prediction
> 0.91
> 0.13
> 0.78
> ...
> ```

In [90]:
import pandas as pd
from sklearn.metrics import accuracy_score

# ── Load best checkpoint for inference ────────────────────────────────────────
best_model = SentinelNet(input_dim=INPUT_DIM)
best_model.eval()

# ── Generate predictions on the test set ──────────────────────────────────────
with torch.no_grad():
    final_probs = best_model(X_test_t).numpy().flatten()

# ── Compute local metrics ─────────────────────────────────────────────────────
final_preds    = (final_probs >= 0.5).astype(int)
true_labels    = y_test_t.numpy().flatten().astype(int)
submission_acc = accuracy_score(true_labels, final_preds)
submission_auc = compute_auc(true_labels, final_probs)

# ── Save to predictions.csv ───────────────────────────────────────────────────
pred_df = pd.DataFrame({"prediction": final_probs})
pred_df.to_csv("predictions.csv", index=False)

print("=" * 50)
print("        PREDICTIONS EXPORTED")
print("=" * 50)
print(f"  Rows saved   : {len(pred_df)}")
print(f"  File         : predictions.csv")
print(f"  Accuracy     : {submission_acc:.4f}  ({submission_acc*100:.2f}%)")
print(f"  ROC-AUC      : {submission_auc:.4f}")
print("=" * 50)
print()
print("  📤 Submit predictions.csv on the challenge platform.")
print("  The judge will evaluate your ROC-AUC automatically.")
print()
print("  Required : ROC-AUC ≥ 0.85")
print("  If you pass → 🏴 Flag is revealed on the platform.")

pred_df.head(10)

        PREDICTIONS EXPORTED
  Rows saved   : 209
  File         : predictions.csv
  Accuracy     : 0.4976  (49.76%)
  ROC-AUC      : 0.5065

  📤 Submit predictions.csv on the challenge platform.
  The judge will evaluate your ROC-AUC automatically.

  Required : ROC-AUC ≥ 0.85
  If you pass → 🏴 Flag is revealed on the platform.


,prediction
0,0.509408
1,0.511648
2,0.510213
3,0.511206
4,0.510301
5,0.510019
6,0.511217
7,0.510855
8,0.511369
9,0.511814


## 📤 Submit Your Predictions

Once `predictions.csv` has been saved, run the cell below.

It will send your predictions to the evaluation server. The server holds the true test labels internally and computes the ROC-AUC and accuracy automatically — your CSV is never compared client-side.

If your ROC-AUC is **≥ 0.85**, the flag is revealed instantly. Otherwise you will see your current scores so you know how close you are.

> 🔒 **Do not modify the code cell below.** It handles the submission to the evaluation server — run it as-is.

In [91]:
import requests
import os

# ── Configuration ─────────────────────────────────────────────────────────────
API_URL  = "http://localhost:8000/challenge-4"
CSV_FILE = "predictions.csv"

# ── Submit ────────────────────────────────────────────────────────────────────
if not os.path.exists(CSV_FILE):
    print(f"❌ '{CSV_FILE}' not found. Run Step 9 first to generate predictions.")
else:
    with open(CSV_FILE, "rb") as f:
        response = requests.post(API_URL, files={"file": (CSV_FILE, f, "text/csv")})

    result = response.json()

    print("=" * 50)
    if result.get("result") == "success":
        print("✅  CHALLENGE SOLVED!")
        print(f"🏴  Flag     : {result['flag']}")
    else:
        print("❌  Not quite right. Keep debugging!")

    print(f"\n💬  {result.get('message', '')}")
    print(f"\n📊  Scores:")
    print(f"    ROC-AUC  : {result.get('roc_auc',  'N/A')}")
    print(f"    Accuracy : {result.get('accuracy', 'N/A')}")
    print("=" * 50)

❌  Not quite right. Keep debugging!

💬  The Sentinel is still struggling. ROC-AUC: 0.4935 (need ≥ 0.85). Accuracy: 0.4976. Keep debugging the pipeline.

📊  Scores:
    ROC-AUC  : 0.4935
    Accuracy : 0.4976
